# 内容：依据特征树构造新特征

In [1]:
# 导包
import re
import csv
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 将正类和负类数据抽取成list
def extract_data(path):
    lines_list =[]
    with open(path, 'r') as f:
        for line in f.readlines():
            lines_list.append(line)
    return lines_list

In [3]:
# 标准氨基酸 ACDEFGHIKLMNPQRSTVWY
# 当特征为连续k个时，创建k的n次方个特征返回
def create_features(new_features,k,if_gap=False, temp = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] ):
    if if_gap==False:
        row_features = temp
    else:
        row_features = ['*']
    length = len(new_features)
    features = []
    for i in range(length):
        for j in row_features:
            features.append(new_features[i] + j)
    if k > 1:
        k -= 1
        features = create_features(features,k)
    return features

In [4]:
def get_ratio(feature, data, pat= '[A-Z]'):
    # 获取某一个特征在某一类（正类或负类）中的比率
    length = len(data)
    freq = 0
    for d in data:
        pattern = r''
        for s in feature:
            if s!='*':
                pattern += s
            else:
                pattern += pat   # 修改
        flag = re.search(pattern, d)
        if flag != None:
            freq += 1
    return freq/length

In [5]:
def get_class_ratio(features, data, pat):
    # 获取一类在所有样本中的比率
    features_ratio = {}
    for feat in features:
        features_ratio[feat] = get_ratio(feat, data,pat)
    return features_ratio  # 字典

In [6]:
def get_features(psoitive_output, ratio = 0.75):
    temp = []
    for i in psoitive_output.columns:
        if psoitive_output.loc['psoitive_ratio',i] <ratio:
            temp.append(i)
    return temp

In [7]:
# 特征树
def get_tree_features(features, gap, depth=0, stop_depth=27, temp=None, pat=None):
    global number # 当前生成的特征是第几个，用于输出特征
    depth += 1    # 当前特征树的深度
    print(depth)
    if depth>stop_depth:  # 迭代终止条件，kmer的长度
        return None
    if gap == False: # 当前生成特征不是*
        row_features = create_features(features, 1, gap, temp) # 生成新的特征集合
        #print(row_features)
        psoitive_ratio = get_class_ratio(row_features, positive_data, pat) # 获取特征集的百分比
        psoitive_output = pd.DataFrame([psoitive_ratio],index = ['psoitive_ratio']) # 生成dataframe
        drop_feature = get_features(psoitive_output)  # 生成不满足条件的特征
        psoitive_output.drop(drop_feature, axis = 1, inplace = True) # 将不满足条件的特征去除
        if psoitive_output.empty:     # 迭代终止条件，该特征没有
            print('该序列特征为空')
            return None
        else:
            psoitive_output = psoitive_output.sort_values('psoitive_ratio',axis = 1,ascending=False)
            negative_ratio = get_class_ratio(psoitive_output.columns, negative_data, pat)
            negative_output = pd.DataFrame([negative_ratio],index = ['negative_ratio'])
            output = pd.concat([psoitive_output,negative_output],axis = 0)
            #print(output)
            output = output.sort_values('negative_ratio',axis = 1)
            number += 1
            train_data_path_part = train_data_path_part1 + str(number) + train_data_path_part3
            if len(output.columns) > 100:
                output = output.iloc[:,:100]
                #print(output)
                output.to_csv(train_data_path_part, index = None)
            else:
                #print(output)
                output.to_csv(train_data_path_part, index = None)
            features = output.columns
            get_tree_features(features, False, depth, stop_depth, temp, pat)
            get_tree_features(features, True, depth, stop_depth, temp, pat)
            
    else:
        row_features = create_features(features, 1, gap)
        #print(row_features)
        get_tree_features(row_features, False, depth, stop_depth, temp, pat)
        get_tree_features(row_features, True, depth, stop_depth, temp, pat)

In [8]:
# path
path_data_dir = os.path.dirname(os.path.dirname(os.getcwd()))

path_new_positive = path_data_dir + r'\new_data\Cancerlectin\merge\positive.csv'
path_new_negative = path_data_dir + r'\new_data\Cancerlectin\merge\negitive.csv'

train_data_path_part1 = path_data_dir + r'\new_data\Cancerlectin\merge_gap\feature_select_'
train_data_path_part2 = '_'
train_data_path_part3 = r'_gap.csv'

In [9]:
# 抽取正负样本数据
positive_data = extract_data(path_new_positive)
negative_data = extract_data(path_new_negative)

In [10]:
number = 0

In [11]:
low = 1000
for i in positive_data:
    if len(i)<low:
        low = len(i)

In [ ]:
#temp = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
#pat = '[A-Z]'

temp = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']
pat = '[a-m]'

#temp = ['1','2','3','4']
#pat = '[1-4]'

get_tree_features([''],gap=False, depth=0, stop_depth = low,temp=temp, pat=pat)

1
2
3
4
该序列特征为空
4
5
该序列特征为空
5
6
该序列特征为空
6
7
该序列特征为空
7
8
该序列特征为空
8
9
该序列特征为空
9
10
该序列特征为空
10
11
该序列特征为空
11
12
该序列特征为空
12
13
该序列特征为空
13
14
该序列特征为空
14
15
该序列特征为空
15
16
该序列特征为空
16
17
该序列特征为空
17
18
该序列特征为空
18
19
该序列特征为空
19
20
该序列特征为空
20
21
该序列特征为空
21
22
该序列特征为空
22
23
该序列特征为空
23
24
该序列特征为空
24
25
该序列特征为空
25
26
该序列特征为空
26
27
该序列特征为空
27
28
该序列特征为空
28
29
该序列特征为空
29
30
该序列特征为空
30
31
该序列特征为空
31
32
该序列特征为空
32
33
该序列特征为空
33
34
该序列特征为空
34
35
该序列特征为空
35
36
该序列特征为空
36
37
该序列特征为空
37
38
该序列特征为空
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47
48
该序列特征为空
48
49
该序列特征为空
49
50
该序列特征为空
50
51
该序列特征为空
51
52
该序列特征为空
52
53
该序列特征为空
53
54
54
3
4
5
该序列特征为空
5
6
该序列特征为空
6
7
该序列特征为空
7
8
该序列特征为空
8
9
该序列特征为空
9
10
该序列特征为空
10
11
该序列特征为空
11
12
该序列特征为空
12
13
该序列特征为空
13
14
该序列特征为空
14
15
该序列特征为空
15
16
该序列特征为空
16
17
该序列特征为空
17
18
该序列特征为空
18
19
该序列特征为空
19
20
该序列特征为空
20
21
该序列特征为空
21
22
该序列特征为空
22
23
该序列特征为空
23
24
该序列特征为空
24
25
该序列特征为空
25
26
该序列特征为空
2

该序列特征为空
53
54
54
14
15
16
该序列特征为空
16
17
该序列特征为空
17
18
该序列特征为空
18
19
该序列特征为空
19
20
该序列特征为空
20
21
该序列特征为空
21
22
该序列特征为空
22
23
该序列特征为空
23
24
该序列特征为空
24
25
该序列特征为空
25
26
该序列特征为空
26
27
该序列特征为空
27
28
该序列特征为空
28
29
该序列特征为空
29
30
该序列特征为空
30
31
该序列特征为空
31
32
该序列特征为空
32
33
该序列特征为空
33
34
该序列特征为空
34
35
该序列特征为空
35
36
该序列特征为空
36
37
该序列特征为空
37
38
该序列特征为空
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47
48
该序列特征为空
48
49
该序列特征为空
49
50
该序列特征为空
50
51
该序列特征为空
51
52
该序列特征为空
52
53
该序列特征为空
53
54
54
15
16
17
该序列特征为空
17
18
该序列特征为空
18
19
该序列特征为空
19
20
该序列特征为空
20
21
该序列特征为空
21
22
该序列特征为空
22
23
该序列特征为空
23
24
该序列特征为空
24
25
该序列特征为空
25
26
该序列特征为空
26
27
该序列特征为空
27
28
该序列特征为空
28
29
该序列特征为空
29
30
该序列特征为空
30
31
该序列特征为空
31
32
该序列特征为空
32
33
该序列特征为空
33
34
该序列特征为空
34
35
该序列特征为空
35
36
该序列特征为空
36
37
该序列特征为空
37
38
该序列特征为空
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47

该序列特征为空
52
53
该序列特征为空
53
54
54
34
35
36
该序列特征为空
36
37
该序列特征为空
37
38
该序列特征为空
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47
48
该序列特征为空
48
49
该序列特征为空
49
50
该序列特征为空
50
51
该序列特征为空
51
52
该序列特征为空
52
53
该序列特征为空
53
54
54
35
36
37
该序列特征为空
37
38
该序列特征为空
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47
48
该序列特征为空
48
49
该序列特征为空
49
50
该序列特征为空
50
51
该序列特征为空
51
52
该序列特征为空
52
53
该序列特征为空
53
54
54
36
37
38
该序列特征为空
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47
48
该序列特征为空
48
49
该序列特征为空
49
50
该序列特征为空
50
51
该序列特征为空
51
52
该序列特征为空
52
53
该序列特征为空
53
54
54
37
38
39
该序列特征为空
39
40
该序列特征为空
40
41
该序列特征为空
41
42
该序列特征为空
42
43
该序列特征为空
43
44
该序列特征为空
44
45
该序列特征为空
45
46
该序列特征为空
46
47
该序列特征为空
47
48
该序列特征为空
48
49
该序列特征为空
49
50
该序列特征为空
50
51
该序列特征为空
51
52
该序列特征为空
52
53
该序列特征为空
53
54
